In [ ]:
import geopandas as gpd
import os
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')
    
    conn.autocommit = True
    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def insert_data(cur, values):
    query = f" \
        INSERT INTO public.municipios (cod_ibge, id_estado, nome, geom) VALUES ( \
            {values[0]}, \
            {values[1]}, \
            '{values[2]}', \
            ST_GeomFromText('{values[3]}', 4326) \
        );"
    
    cur.execute(query)

---

In [ ]:
path = '../../arquivos/ibge/municipios'

In [ ]:
folders = os.listdir(path)

In [ ]:
conn, cur = open_connection()

In [ ]:
for folder in folders:
    for file in os.listdir(f"{path}/{folder}"):
        if file.split('.')[-1] == 'shp':
            gdf = gpd.read_file(f"{path}/{folder}/{file}")
            
            for row in gdf.itertuples():
                name = row[3].replace("'", "''")
                values = [int(row[2]), int(row[2][:2]), name, row[4]]
                
                insert_data(cur, values)

In [ ]:
close_connection(conn, cur)